In [3]:
import geopandas as gpd
import pandas as pd
from shapely import wkb

pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

linha_cores = {
    "VERMELHA": "#ED1C24",   # Linha 3 - Vermelha
    "VERDE": "#00A859",      # Linha 2 - Verde
    "AZUL": "#0072BC",       # Linha 1 - Azul
    "AMARELA": "#FFD700",    # Linha 4 - Amarela
    "LILAS": "#9B59B6",      # Linha 5 - Lilás
    "PRATA": "#A7A9AC",      # Linha 15 - Prata (Monotrilho)
    "JADE": "#00A86B",       # Linha 13 - Jade
    "DIAMANTE": "#7D7F7D",   # Linha 9 - Diamante
    "ESMERALDA": "#00A99D",  # Linha 9 (antiga) / 10 Esmeralda
    "CORAL": "#FF7F50",      # Linha 11 - Coral
    "SAFIRA": "#0066CC",     # Linha 12 - Safira
    "RUBI": "#E0115F",       # Linha 7 - Rubi
    "TURQUESA": "#30D5C8",   # Linha 10/14 futura - Turquesa
    "OURO": "#D4AF37",       # Linha 17 - Ouro
    "LARANJA": "#F7931E",    # Linha 6 - Laranja (em obras)
    "VIOLETA": "#8A2BE2",    # Proposta/futura
    "CELESTE": "#5DADEC",    # Futuras expansões
    "ROSA": "#FF69B4",       # Linha 20 - Rosa (planejada)
    "MARROM": "#8B4513",     # Planejada
    "ALPHAVILLE - CAMPO LIMPO": "#964B00", # sugestão (tonalidade marrom/terra)
    "ONIX": "#353839"        # sugestão (cor preta fosca)
}

In [4]:
# Lendo os arquivos parquet
fare_attributes_historico = pd.read_parquet("dados/fare_attributes_historico.parquet")
fare_rules_historico = pd.read_parquet("dados/fare_rules_historico.parquet")
frequencies_historico_categorizado = pd.read_parquet("dados/frequencies_historico_categorizado.parquet")
frequencies_historico = pd.read_parquet("dados/frequencies_historico.parquet")
passageiros_historico_frequencia_ideal = pd.read_parquet("dados/passageiros_historico_frequencia_ideal.parquet")
passageiros_historico_pivot = pd.read_parquet("dados/passageiros_historico_pivot.parquet")
passageiros_unificado = pd.read_parquet("dados/passageiros_unificado.parquet")
rotas_historico_categorizado = pd.read_parquet("dados/rotas_historico_categorizado.parquet")
routes_dist_km_historico = pd.read_parquet("dados/routes_dist_km_historico.parquet")
routes_historico = pd.read_parquet("dados/routes_historico.parquet")
stop_times_historico = pd.read_parquet("dados/stop_times_historico.parquet")
stops_historico_categorizado = pd.read_parquet("dados/stops_historico_categorizado.parquet")
stops_historico = pd.read_parquet("dados/stops_historico.parquet")
trips_historico = pd.read_parquet("dados/trips_historico.parquet")

In [5]:
estacoes_metro_e_trem = pd.read_parquet("dados/estacoes_metro_e_trem.parquet")
estacoes_metro_e_trem['cor']=estacoes_metro_e_trem['nm_linha_metro_trem'].map(linha_cores)
# converter WKB para geometria shapely
if estacoes_metro_e_trem["geometry"].dtype == object:  # caso esteja em bytes
    estacoes_metro_e_trem["geometry"] = estacoes_metro_e_trem["geometry"].apply(lambda x: wkb.loads(x))

gdf_stations = gpd.GeoDataFrame(estacoes_metro_e_trem, geometry="geometry", crs="EPSG:31983")
gdf_stations = gdf_stations.to_crs(epsg=4326)

# extrai lon/lat
gdf_stations["lon"] = gdf_stations.geometry.x
gdf_stations["lat"] = gdf_stations.geometry.y

In [6]:
gdf_setores_censitarios = gpd.read_file("dados/POPULAÇÃO-CENSO-2022-IBGE/BR_setores_CD2022.zip")
gdf_setores_censitarios=gdf_setores_censitarios[gdf_setores_censitarios.NM_UF=='São Paulo']

cols_v = [f'v000{i}' for i in range(1, 8)]
for col in cols_v:
    gdf_setores_censitarios[col] = pd.to_numeric(gdf_setores_censitarios[col], errors='coerce')

# Plot da rota mais recente

In [7]:
import json
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

def plot_route_on_gdf(
    df,
    route_id,
    data,
    gdf,
    rotas_df,
    df_estacoes_metro_e_trem,
    save=True,
    simplify_tol=0.0002,
    margin_deg=0.02,
    map_zoom=12,
    add_border_layer=False
    ):
    # lista de linhas de trem/metro próximas
    lista_estacoes = rotas_historico_categorizado[
        rotas_historico_categorizado.route_id == route_id
    ].estacoes_proximas.str.upper().apply(lambda x: [y.strip() for y in x.split(",")]).to_list()[0]

    # filtra estacoes do dataset
    gdf_stations_rota = df_estacoes_metro_e_trem[
        gdf_stations['nm_estacao_metro_trem'].isin(lista_estacoes)
    ].copy()

    

    # =====================
    # filtra rota
    df_route = df[(df["route_id"] == route_id) & (df["data_referencia"] == data)].sort_values(
        ["direction_id", "stop_sequence"]
    ).copy()

    if df_route.empty:
        raise ValueError(f"Nenhuma parada encontrada para route_id={route_id} em data_referencia={data}")

    df_route["stop_lon"] = pd.to_numeric(df_route["stop_lon"], errors="coerce")
    df_route["stop_lat"] = pd.to_numeric(df_route["stop_lat"], errors="coerce")
    df_route = df_route.dropna(subset=["stop_lon", "stop_lat"])
    if df_route.empty:
        raise ValueError("Todas as paradas têm lon/lat inválidos depois da conversão.")

    # cor da rota
    cor_hex = "#000000"
    try:
        cor = rotas_df.loc[rotas_df["route_id"] == route_id, "route_color"].to_list()[0]
        cor_hex = "#" + str(cor).lstrip("#")
        if cor_hex == "#None":
            cor_hex = "#000000"
    except Exception:
        pass

    # garante WGS84
    try:
        gdf_wgs = gdf.to_crs(epsg=4326) if gdf.crs is not None else gdf.copy()
    except Exception:
        gdf_wgs = gdf.copy()

    # subset por bbox
    minx = df_route["stop_lon"].min() - margin_deg
    maxx = df_route["stop_lon"].max() + margin_deg
    miny = df_route["stop_lat"].min() - margin_deg
    maxy = df_route["stop_lat"].max() + margin_deg
    try:
        if hasattr(gdf_wgs, "sindex") and gdf_wgs.sindex is not None:
            possible_idx = list(gdf_wgs.sindex.intersection((minx, miny, maxx, maxy)))
            gdf_subset = gdf_wgs.iloc[possible_idx].copy()
        else:
            gdf_subset = gdf_wgs.cx[minx:maxx, miny:maxy].copy()
    except Exception:
        gdf_subset = gdf_wgs.copy()

    if gdf_subset.empty:
        gdf_subset = gdf_wgs.copy()

    if simplify_tol and len(gdf_subset) > 0:
        gdf_subset["geometry"] = gdf_subset.geometry.simplify(simplify_tol, preserve_topology=True)

    geojson_gdf = json.loads(gdf_subset.to_json())

    # traces da rota
    route_trace = go.Scattermapbox(
        lon=df_route["stop_lon"],
        lat=df_route["stop_lat"],
        mode="lines",
        line=dict(width=4, color=cor_hex),
        name=f"Linha {route_id}",
        hoverinfo="none",
    )

    hover_texts = [
        (
            f"<b>Parada:</b> {row.get('stop_name','-')}<br>"
            f"<b>Seq:</b> {row.get('stop_sequence','-')}<br>"
            f"<b>Sentido:</b> {row.get('direction_id','-')}"
        )
        for _, row in df_route.iterrows()
    ]

    stop_trace = go.Scattermapbox(
        lon=df_route["stop_lon"],
        lat=df_route["stop_lat"],
        mode="markers",
        marker=dict(size=9, color=cor_hex, symbol="circle"),
        name=f"Paradas {route_id}",
        hoverinfo="text",
        hovertext=hover_texts,
    )

    # traces das estações de trem/metro
    estacoes_trace = go.Scattermapbox(
    lon=gdf_stations_rota["lon"],
    lat=gdf_stations_rota["lat"],
    mode="markers+text",  # markers visíveis + nome opcional
    marker=dict(
        size=12,
        color=gdf_stations_rota.get("cor", "red"),  # usa coluna cor, ou vermelho se não existir
        symbol="circle"  # "rail" é garantido, "rail-light" pode não renderizar
    ),
    name="Estações Metrô/Trem",
    hoverinfo="text",
    hovertext=gdf_stations_rota["nm_estacao_metro_trem"]+" \nLinha: " +gdf_stations_rota["nm_linha_metro_trem"],
    text=gdf_stations_rota["nm_estacao_metro_trem"],  # opcional: escreve nome direto no mapa
    textposition="top center"  # coloca o nome acima do ponto
    )

    center_lat = float(df_route["stop_lat"].mean())
    center_lon = float(df_route["stop_lon"].mean())

    layer_fill = dict(
    sourcetype="geojson",
    source=geojson_gdf,
    type="fill",
    color="rgba(200,200,200,0.4)",  # mais transparente
    below="traces"  # garante que markers fiquem por cima
    )

    layers = [layer_fill]

    if add_border_layer:
        layer_border = dict(
            sourcetype="geojson",
            source=geojson_gdf,
            type="line",
            color="rgba(0,0,0,0.6)",
            line=dict(width=0.6),
            opacity=1,
        )
        layers.append(layer_border)

    fig = go.Figure(data=[route_trace, stop_trace, estacoes_trace])
    fig.update_layout(
        title=f"Rota {route_id} - {data} sobre Setores Censitários",
        autosize=True,
        showlegend=True,
        hovermode="closest",
        margin=dict(l=0, r=0, t=40, b=0),
        mapbox=dict(
            style="carto-darkmatter",
            center=dict(lat=center_lat, lon=center_lon),
            zoom=map_zoom,
            layers=layers,
        ),
    )

    if save:
        out_path = rf"dados/plots_rotas_mais_recente/rota_{route_id}_com_gdf.html"
        fig.write_html(out_path)
        print(f"Mapa salvo em: {out_path}")

    return fig


In [8]:
trips_historico_sorted = trips_historico.sort_values(
    ["route_id", "data_referencia"], ascending=[True, False]
)

latest_dates = trips_historico_sorted.groupby("route_id")["data_referencia"].first().reset_index()

trips_historico_recent = trips_historico.merge(
    latest_dates, on=["route_id", "data_referencia"], how="inner"
)

In [ ]:
for index,row in trips_historico_recent.drop_duplicates(subset=['route_id','data_referencia']).iterrows():
    try:
        plot_route_on_gdf(trips_historico, row['route_id'],row['data_referencia'],gdf_setores_censitarios,rotas_df=rotas_historico_categorizado, save=True)
    except: print(row['route_id'],row['data_referencia'])

# Plot do histórico das rotas

In [10]:
import plotly.graph_objects as go
import pandas as pd
import json
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def plot_route_history(
    df,
    route_id,
    gdf,
    df_estacoes_metro_e_trem,
    rotas_df,
    save=True,
    simplify_tol=0.0002,
    margin_deg=0.02,
    map_zoom=12,
    add_border_layer=False
    ):
    
    # lista de linhas de trem/metro próximas
    lista_estacoes = rotas_historico_categorizado[
        rotas_historico_categorizado.route_id == route_id
    ].estacoes_proximas.str.upper().apply(lambda x: [y.strip() for y in x.split(",")]).to_list()[0]

    # filtra estacoes do dataset
    gdf_stations_rota = df_estacoes_metro_e_trem[
        df_estacoes_metro_e_trem['nm_estacao_metro_trem'].isin(lista_estacoes)
    ].copy()


    # filtra rota para todos os anos
    df_route = df[df["route_id"] == route_id].copy()
    if df_route.empty:
        raise ValueError(f"Nenhuma parada encontrada para route_id={route_id}")

    # garantir numéricos
    df_route['stop_lon'] = pd.to_numeric(df_route['stop_lon'], errors='coerce')
    df_route['stop_lat'] = pd.to_numeric(df_route['stop_lat'], errors='coerce')
    df_route['ano'] = df_route['data_referencia']
    # pd.to_datetime(
    #     df_route['data_referencia'].astype(str), 
    #     format="%Y%m", errors="coerce"
    #     ).dt.year
    
    df_route = df_route.dropna(subset=['stop_lon', 'stop_lat'])

    if df_route.empty:
        raise ValueError("Todas as paradas têm lon/lat inválidos depois da conversão.")

    # cor base da linha (se disponível)
    cor = "#000000"
    try:
        cor = rotas_df.loc[rotas_df['route_id'] == route_id, 'route_color'].to_list()[0]
        cor = "#" + str(cor).lstrip('#')
        if cor == "#None": cor = "#F614F6"
    except Exception:
        pass

    # garante WGS84
    try:
        gdf_wgs = gdf.to_crs(epsg=4326) if gdf.crs is not None else gdf.copy()
    except Exception:
        gdf_wgs = gdf.copy()

    # bounding box geral
    minx = df_route['stop_lon'].min() - margin_deg
    maxx = df_route['stop_lon'].max() + margin_deg
    miny = df_route['stop_lat'].min() - margin_deg
    maxy = df_route['stop_lat'].max() + margin_deg
    try:
        if hasattr(gdf_wgs, 'sindex') and gdf_wgs.sindex is not None:
            possible_idx = list(gdf_wgs.sindex.intersection((minx, miny, maxx, maxy)))
            gdf_subset = gdf_wgs.iloc[possible_idx].copy()
        else:
            gdf_subset = gdf_wgs.cx[minx:maxx, miny:maxy].copy()
    except Exception:
        gdf_subset = gdf_wgs.copy()

    if gdf_subset.empty:
        gdf_subset = gdf_wgs.copy()

    if simplify_tol and len(gdf_subset) > 0:
        gdf_subset['geometry'] = gdf_subset.geometry.simplify(simplify_tol, preserve_topology=True)

    geojson_gdf = json.loads(gdf_subset.to_json())

    # gera paleta de cores para anos
    anos = sorted(df_route['ano'].unique())
    n = len(anos)
    # # norm = mcolors.Normalize(vmin=min(anos), vmax=max(anos))
    # cmap = cm.get_cmap("gist_rainbow", len(anos))
    colors = gerar_degrade_cor(cor, n)

    traces = []
    for i, ano in enumerate(anos):
        df_year = df_route[df_route['ano'] == ano].sort_values(["direction_id","stop_sequence"])
        # color = mcolors.to_hex(cmap(i))
        color = colors[i]

        route_trace = go.Scattermapbox(
            lon=df_year["stop_lon"],
            lat=df_year["stop_lat"],
            mode="lines",
            line=dict(width=4, color=color),
            name=f"{ano}",
            hoverinfo="none"
        )

        traces.append(route_trace)

    # centro do mapa
    center_lat = float(df_route["stop_lat"].mean())
    center_lon = float(df_route["stop_lon"].mean())

    # traces das estações de trem/metro
    estacoes_trace = go.Scattermapbox(
    lon=gdf_stations_rota["lon"],
    lat=gdf_stations_rota["lat"],
    mode="markers+text",  # markers visíveis + nome opcional
    marker=dict(
        size=12,
        color=gdf_stations_rota.get("cor", "red"),  # usa coluna cor, ou vermelho se não existir
        symbol="circle"  # "rail" é garantido, "rail-light" pode não renderizar
    ),
    name="Estações Metrô/Trem",
    hoverinfo="text",
    hovertext=gdf_stations_rota["nm_estacao_metro_trem"]+" linha " +gdf_stations_rota["nm_linha_metro_trem"] +" "+ gdf_stations_rota["tipo"] +" "+ gdf_stations_rota["existencia"],
    text=gdf_stations_rota["nm_estacao_metro_trem"],  # opcional: escreve nome direto no mapa
    textposition="top center"  # coloca o nome acima do ponto
    )

    traces.append(estacoes_trace)

    layer_fill = dict(
    sourcetype="geojson",
    source=geojson_gdf,
    type="fill",
    color="rgba(200,200,200,0.4)",  # mais transparente
    below="traces"  # garante que markers fiquem por cima
    )

    layers = [layer_fill]
    if add_border_layer:
        layer_border = dict(
            sourcetype="geojson",
            source=geojson_gdf,
            type="line",
            color="rgba(0,0,0,0.6)",
            line=dict(width=0.6),
            opacity=1
        )
        layers.append(layer_border)



    # monta figura
    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"Histórico da Rota {route_id} (mês a mês)",
        autosize=True,
        showlegend=True,
        hovermode="closest",
        margin=dict(l=0, r=0, t=40, b=0),
        mapbox=dict(
            style="carto-darkmatter",
            center=dict(lat=center_lat, lon=center_lon),
            zoom=map_zoom,
            layers=layers
        )
    )

    if save:
        out_path = rf"dados/plots_rotas_historico/rota_{route_id}_historico.html"
        fig.write_html(out_path)
        # print(f"Mapa salvo em: {out_path}")

    return fig


In [11]:
import colorsys
import matplotlib.colors as mcolors

def gerar_degrade_cor(base_hex, n, fator_claro=1.7, fator_escuro=0.3):
    """
    Gera n cores em degradê a partir de uma cor base.
    
    base_hex : str -> cor base em hexadecimal (ex: "#008000")
    n        : int -> número de cores a gerar
    fator_claro : float -> fator para clarear ( > 1 deixa mais claro)
    fator_escuro: float -> fator para escurecer ( < 1 deixa mais escuro)
    """
    # converte hex para RGB [0,1]
    r, g, b = mcolors.to_rgb(base_hex)
    
    # converte para HLS
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    
    # define os limites claro/escuro da luminosidade
    l_min = max(0, l * fator_escuro)
    l_max = min(1, l * fator_claro)
    
    # gera variações lineares
    degradê = [
        mcolors.to_hex(colorsys.hls_to_rgb(h, l_min + (l_max - l_min) * i/(n-1), s))
        for i in range(n)
    ]
    
    return degradê

In [12]:
for index,row in trips_historico.drop_duplicates(subset=['route_id']).iterrows():
    try:
        plot_route_history(trips_historico, row['route_id'],gdf_setores_censitarios,gdf_stations,rotas_historico_categorizado,map_zoom=12, save=True)
    except: print("Erro em ", row['route_id'])

Erro em  119P10
Erro em  172121
Erro em  372F10
Erro em  737G10
Erro em  CPTM L07
Erro em  CPTM L08
Erro em  CPTM L09
Erro em  CPTM L10
Erro em  CPTM L11
Erro em  CPTM L12
Erro em  METRÔ L1
Erro em  METRÔ L2
Erro em  METRÔ L3
Erro em  METRÔ L4
Erro em  METRÔ L5
Erro em  561210
Erro em  METRÔ 15
Erro em  233F10
Erro em  341421
Erro em  473521
Erro em  633821
Erro em  647521
Erro em  701U21
Erro em  CPTM L13
Erro em  909N10
Erro em  808U10
Erro em  700421
Erro em  405010
Erro em  606F1
Erro em  879A10
